In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [2]:
pip install pydantic

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install langchain openai langchain-openai langchain-community

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 27.0 MB/s eta 0:00:00

   ---------- ----------------------------- 2/8 [requests]
   ---------- ----------------------------- 2/8 [requests]
   ---------- ----------------------------- 2/8 [requests]
   ---------- ----------------------------- 2/8 [requests]
   --------------- ------------------------ 3/8 [marshmallow]
   --------------- ------------------------ 3/8 [marshmallow]
   -------------------- ------------------- 4/8 [httpx-sse]
   ------------------------- -------------- 5/8 [dataclasses-json]
   ------------------------------ --------- 6/8 [pydantic-settings]
   ------------------------------ --------- 6/8 [pydantic-settings]
   ------------------------------ --------- 6/8 [pydantic-settings]
   ------------------------------ --------- 6/8 [pydantic-settings]
   -------------

In [2]:
from langchain_classic.document_loaders import GitLoader

def file_filter(file_path):
    return file_path.endswith(".md")

loader=GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs=loader.load()
print(len(raw_docs))

36


## Document transformers

In [3]:
from langchain_classic.text_splitter import CharacterTextSplitter

text_splitter=CharacterTextSplitter(chunk_size=1000,chunk_overlap=0)

docs=text_splitter.split_documents(raw_docs)
print(len(docs))

Created a chunk of size 1367, which is longer than the specified 1000
Created a chunk of size 1455, which is longer than the specified 1000
Created a chunk of size 1299, which is longer than the specified 1000


96


In [4]:
from langchain_openai import OpenAIEmbeddings

embeddings=OpenAIEmbeddings()

In [5]:
pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
query="AWS의 S3에서 데이터를 읽기 위한 DocumentLoader가 있나요?"

vector=embeddings.embed_query(query)

print(len(vector))
print(vector[:100])

1536
[-0.01556143257766962, -0.012867593206465244, 0.021737877279520035, -0.01614966429769993, -0.024117546156048775, 0.045186977833509445, -0.019705800339579582, -0.0077606625854969025, -0.0040641543455421925, 0.022138945758342743, 0.01052134670317173, 0.01549458783119917, 0.0010402697371318936, -0.018796712160110474, -0.007393017411231995, -0.002800790360197425, 0.0177271980792284, -0.00953204557299614, -0.009244613349437714, -0.03665760159492493, 0.012967860326170921, -0.015628276392817497, -0.005621634423732758, 0.0017429740400984883, -0.0026554034557193518, 0.009017341770231724, 0.007687133736908436, -0.04016026109457016, 0.007219220977276564, 0.0026470478624105453, 0.0526735782623291, -0.013041389174759388, -0.0068415491841733456, -0.022727178409695625, -0.010026696138083935, -0.014705820940434933, 0.00838231761008501, -0.0006972731789574027, -0.006173102650791407, -0.02152397483587265, 0.002012023702263832, -0.006493956781923771, -0.0006116284639574587, -0.04347575455904007, 0.0

In [7]:
pip install chromadb

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/21.4 MB ? eta -:--:--
   ------- -------------------------------- 3.9/21.4 MB 19.8 MB/s eta 0:00:01
   -------------- ------------------------- 7.6/21.4 MB 19.3 MB/s eta 0:00:01
   --------------------------- ------------ 14.9/21.4 MB 24.8 MB/s eta 0:00:01
   ---------------------------------------  21.0/21.4 MB 25.8 MB/s eta 0:00:01
   ---------------------------------------- 21.4/21.4 MB 24.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 37.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.5 MB ? eta -:--:--
   ----------------- ---------------------- 6.0/13.5 MB 30.5 MB/s eta 0:00:01
   ----------------------------------- ---- 11.8/13.5 MB 31.6 MB/s eta 0:00:01
   ---------------------------------------- 13.5/13.5 MB 23.2 MB/s eta 0:00:00
   ----

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t

In [8]:
from langchain_classic.vectorstores import Chroma

db=Chroma.from_documents(docs,embeddings)

## Retriever

In [10]:
retriever=db.as_retriever()

In [11]:
query="AWS S3에서 데이터를 불러올수있는 DocumentLoader가 있나요?"
context_docs=retriever.invoke(query)
print(f"eln ={len(context_docs)}")

first_doc=context_docs[0]
print(f"metadata ={ first_doc.metadata}")
print(first_doc.page_content)

eln =4
metadata ={'file_type': '.md', 'file_name': 'README.md', 'source': '.devcontainer\\README.md', 'file_path': '.devcontainer\\README.md'}
Then you will have a local cloned repo where you can contribute and then create pull requests.

If you already have VS Code and Docker installed, you can use the button above to get started. This will use VSCode to automatically install the Dev Containers extension if needed, clone the source code into a container volume, and spin up a dev container for use.

Alternatively you can also follow these steps to open this repo in a container using the VS Code Dev Containers extension:

1. If this is your first time using a development container, please ensure your system meets the pre-reqs (i.e. have Docker installed) in the [getting started steps](https://aka.ms/vscode-remote/containers/getting-started).

2. Open a locally cloned copy of the code:


## RetrievalQA(Chain)

In [12]:
from langchain_classic.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model_name="gpt-4.1-mini",temperature=0)
qa_chain=RetrievalQA.from_chain_type(llm=llm,chain_type='stuff',retriever=retriever)

qa_chain.invoke(query)

{'query': 'AWS S3에서 데이터를 불러올수있는 DocumentLoader가 있나요?',
 'result': '네, Langchain에는 AWS S3에서 데이터를 불러올 수 있는 DocumentLoader가 있습니다.\n\nLangchain의 `langchain.document_loaders` 모듈 내에 `S3FileLoader`라는 클래스가 있어, AWS S3 버킷에 저장된 파일을 로드할 수 있습니다.\n\n사용 예시는 다음과 같습니다:\n\n```python\nfrom langchain.document_loaders import S3FileLoader\n\n# S3 버킷 내 파일 경로 지정 (예: \'s3://bucket-name/path/to/file.txt\')\ns3_path = "s3://your-bucket-name/path/to/your-file.txt"\n\nloader = S3FileLoader(s3_path)\ndocuments = loader.load()\n```\n\n이때 AWS 자격증명(AWS Access Key, Secret Key 등)은 환경 변수나 AWS CLI 설정을 통해 미리 설정되어 있어야 합니다.\n\n더 자세한 내용과 지원되는 파일 형식은 Langchain 공식 문서의 [통합 문서](https://docs.langchain.com/oss/python/integrations/providers)에서 확인할 수 있습니다.'}